In [27]:
"""Capacitated Vehicle Routing Problem with Time Windows (CVRPTW).
"""
# Import packages
from __future__ import print_function
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In [28]:
"""Distance Matrix from PDV file.
"""
from __future__ import division
from __future__ import print_function
import requests
import json
import urllib
import urllib.request
import pandas as pd

## Global Variables

In [29]:
def build_config():
    config = {}

    config["API_key"] = 'INSERT API KEY'

    #PDV file
    config["PDV_file"] = {}
    config["PDV_file"]["filepath"] = "visitas.csv"
    config["PDV_file"]["sep"]=';'
    config["PDV_file"]["decimal"]=','
    config["PDV_file"]["dtype"]='float'

    # Number of items required per location
    config["demands"] = [0, 0, 1, 2, 1, 2, 4, 8, 8, 1, 2, 1]
    
    # Time taken by each demand unit
    config["time_per_demand_unit"] = 5

    # Array with the number of visits required per location
    config["visits"] = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

    # Dictionary with location names
    config["dict_place_names"] = {0: "Home P1", 1: "Home P2", 2: "PDV_2", 3: "PDV_3", 4: "PDV_4", 
                                  5: "PDV_5", 6: "PDV_6", 7: "PDV_7", 8: "PDV_8", 9: "PDV_9", 
                                  10: "PDV_10", 11: "PDV_11"}

    # Dictionary with promotors names
    config["dict_promotors_names"] = {0: "Promotor_1", 1: "Promotor_2"}

    # Array with promotors starting nodes
    config["promotors_starting_nodes"] = [0, 1]

    # Number of days to be routed
    config["routed_days"] = 5

    # Avg speed
    # (50km/h * 1000m/km)
    config["vehicle_speed"] = 50 * 1000 

    # Working hours per day
    config["horizon"] = 100
    
    return config

## Create Distance Matrix from PDV file

In [30]:
###########################
# Import PDV file #
###########################
def import_pdv_file(PDV_file):
    # importante informacao separado por ponto e virgula e trocar ponto é no decimal
    pdv_df = pd.read_csv(PDV_file["filepath"], 
                         sep=PDV_file["sep"],
                         decimal=PDV_file["decimal"],
                         dtype=PDV_file["dtype"]) 
    
    # separação dos dados de interesse
    #separa os 12 primeiros das colunas Longitude e Latitude
    pdv_df=pdv_df[['Longitude','Latitude'] ][0:12]
    return pdv_df

In [31]:
def create_distance_matrix(pdv_df, API_key):
  addresses = pdv_df[['Longitude','Latitude'] ]
  # Distance Matrix API only accepts 100 elements per request, so get rows in multiple requests.
  max_elements = 100
  num_addresses = len(addresses)
  # Maximum number of rows that can be computed per request.
  max_rows = max_elements // num_addresses
  # num_addresses = q * max_rows + r
  q, r = divmod(num_addresses, max_rows)
  dest_addresses = addresses
  distance_matrix = []
  # Send q requests, returning max_rows rows per request.
  for i in range(q):
    origin_addresses = addresses[i * max_rows: (i + 1) * max_rows]
    # é preciso mudar de formato
    origin_addresses = pd.DataFrame(origin_addresses)
    # mudança de indice
    origin_addresses.index = range(len(origin_addresses))
    response = send_request(origin_addresses, dest_addresses, API_key)
    distance_matrix += build_distance_matrix(response)

  # Get the remaining remaining r rows, if necessary.
  if r > 0:
    origin_addresses = addresses[q * max_rows: q * max_rows + r]
    # é preciso mudar de formato
    origin_addresses = pd.DataFrame(origin_addresses) 
    # mudança de indice
    origin_addresses.index = range(len(origin_addresses))
    # envia para a próxima caixa definida
    response = send_request(origin_addresses, dest_addresses, API_key) 
    distance_matrix += build_distance_matrix(response)
  return distance_matrix

In [32]:
def send_request(origin_addresses, dest_addresses, API_key):
  """ Build and send request for the given origin and destination addresses."""
  def build_address_str(addresses):
    # Build a pipe-separated string of addresses
    address_str = ''
    for i in range(len(addresses) - 1):
        address_str += str(addresses['Latitude'][i]) + ',' + str(addresses['Longitude'][i])+ '|'
    
    address_str += str(addresses['Latitude'][len(addresses) - 1]) + ',' + str(addresses['Longitude'][len(addresses) - 1])
    return address_str

  request = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=metric'
  origin_address_str = build_address_str(origin_addresses)
  dest_address_str = build_address_str(dest_addresses)
  request = request + '&origins=' + origin_address_str + '&destinations=' + \
                       dest_address_str + '&key=' + API_key

  
  with urllib.request.urlopen(request) as url:
    jsonResult = url.read()
  response = json.loads(jsonResult)

  return response

In [33]:
def build_distance_matrix(response):
  distance_matrix = []
  for row in response['rows']:
    row_list = [row['elements'][j]['distance']['value'] for j in range(len(row['elements']))]
    distance_matrix.append(row_list)
  return distance_matrix

## Routing

In [34]:
# Function to add revisiting nodes to the distance matrix, demands, dict_place_names and 
# revists list (to build same vehicle constraint)
def compute_revisits(data, dict_place_names, _distances, visits, demands):
  """Converts node number to take into account revisiting."""
  max_num = data["num_unique_locations"]
  revisits = []
  if data["num_locations"] == data["num_unique_locations"]:
    pass
  else:
    for node_index, number_of_visits in enumerate(visits):
      if number_of_visits > 1:
        revisits_list = [node_index]
        for x in range(number_of_visits - 1):
          dict_place_names[max_num] = dict_place_names[node_index]
          _distances.append(_distances[node_index])
          for distance_list in _distances:
            distance_list.append(distance_list[node_index])
          demands.append(demands[node_index])
          revisits_list.append(max_num)
          max_num += 1
        if number_of_visits == 2:
          revisits.append(revisits_list)
        elif number_of_visits == 3:
          revisits.append([revisits_list[0], revisits_list[1]])
          revisits.append([revisits_list[0], revisits_list[2]])
          revisits.append([revisits_list[1], revisits_list[2]])
        elif number_of_visits == 4:
          revisits.append([revisits_list[0], revisits_list[1]])
          revisits.append([revisits_list[0], revisits_list[2]])
          revisits.append([revisits_list[0], revisits_list[3]])
          revisits.append([revisits_list[1], revisits_list[2]])
          revisits.append([revisits_list[1], revisits_list[3]])
          revisits.append([revisits_list[2], revisits_list[3]])
        elif number_of_visits == 5:
          revisits.append([revisits_list[0], revisits_list[1]])
          revisits.append([revisits_list[0], revisits_list[2]])
          revisits.append([revisits_list[0], revisits_list[3]])
          revisits.append([revisits_list[0], revisits_list[4]])
          revisits.append([revisits_list[1], revisits_list[2]])
          revisits.append([revisits_list[1], revisits_list[3]])
          revisits.append([revisits_list[1], revisits_list[4]])
          revisits.append([revisits_list[2], revisits_list[3]])
          revisits.append([revisits_list[2], revisits_list[4]])
          revisits.append([revisits_list[3], revisits_list[4]])
  return revisits, dict_place_names, _distances, demands

In [35]:
###########################
# Problem Data Definition #
###########################
def create_data_model(distance_matrix, config):
  """Creates the data for the example."""
  data = {}
  # Array of distances between locations.
  #_distances = \
  #        [
  #         [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
  #         [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
  #         [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
  #         [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
  #         [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
  #         [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
  #         [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
  #         [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
  #         [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
  #         [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
  #         [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
  #         [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
  #         [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
  #         [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
  #         [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
  #         [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
  #         [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0]
  #        ]
  _distances = distance_matrix

  # Number of items required per location
  demands = config["demands"]

  # Array with the number of visits required per location
  visits = config["visits"]
  
  # Dictionary with location names
  dict_place_names = config["dict_place_names"]
  
  # Dictionary with promotors names
  dict_promotors_names = config["dict_promotors_names"]
  
  # Array with promotors starting nodes
  promotors_starting_nodes = config["promotors_starting_nodes"]
  
  # Number of days to be routed
  routed_days = config["routed_days"]
  
  # Building array containing each promotor starting node * the number of routed days
  starting_nodes_list = []
  for x in range(len(dict_promotors_names)*routed_days):
    starting_nodes_list.append(promotors_starting_nodes[int(x/routed_days)])

  
  data["num_unique_locations"] = len(_distances)
  data["num_locations"] = sum(visits)
  # We rout one vehicle per promotor * number of routed days
  data["num_vehicles"] = len(dict_promotors_names)*routed_days
  data["depot"] = starting_nodes_list
  # End nodes equals starting nodes
  data["arrive_location"] = data["depot"]
  # Time taken by each demand unit
  data["time_per_demand_unit"] = config["time_per_demand_unit"]
  # Avg speed
  # (50km/h * 1000m/km)
  data["vehicle_speed"] = config["vehicle_speed"]
  # Working hours per day
  data["horizon"] = config["horizon"]
  # Penalty for not visiting a node (very large number - larger than any distance)
  data["penalty"] = 1000000000
  data["dict_promotors_names"] = dict_promotors_names
  data["routed_days"] = routed_days
  
  # Applies compute_revisits function to add revisting nodes to the distance matrix, demands, dict_place_names and
  # revists list (used for building same vehicle constraint) 
  revisits, dict_place_names, _distances, demands = compute_revisits(data, dict_place_names, _distances, visits, demands)
  
  data["revisits"] = revisits
  data["demands"] = demands
  data["distances"] = _distances
  data["dict_place_names"] = dict_place_names
  return data

In [36]:
#######################
# Problem Constraints #
#######################
def create_distance_callback(data):
  """Creates callback to return distance between points."""
  distances = data["distances"]

  def distance_callback(from_node, to_node):
    """Returns the manhattan distance between the two nodes"""
    return distances[from_node][to_node]

  return distance_callback

def create_time_callback(data):
  """Creates callback to get total times between locations."""
  def service_time(node):
    """Gets the service time for the specified location."""
    return data["demands"][node] * data["time_per_demand_unit"]

  def travel_time(from_node, to_node):
    """Gets the travel times between two locations."""
    travel_time = data["distances"][from_node][to_node] / data["vehicle_speed"]
    return travel_time

  def time_callback(from_node, to_node):
    """Returns the total time between the two nodes"""
    serv_time = service_time(from_node)
    trav_time = travel_time(from_node, to_node)
    return serv_time + trav_time

  return time_callback

def add_time_constraints(routing, data, time_callback):
  """Add time constraints."""
  time = "Time"
  routing.AddDimension(
    time_callback,
    data["horizon"], # allow waiting time
    data["horizon"], # maximum time per vehicle
    False, # Don't force start cumul to zero. This doesn't have any effect in this example,
           # since the depot has a start window of (0, 0).
    time)

# For each revisiting node pair, add constraint to prevent from getting the same vehicle (promotor * day)
def add_revisit_constraints(routing, data):
  """Add revisit constraints."""
  for pair_revisit in data["revisits"]:
    constraintActive = routing.ActiveVar(routing.NodeToIndex(pair_revisit[0])) * routing.ActiveVar(routing.NodeToIndex(pair_revisit[1]))
    routing.solver().Add(
    constraintActive * routing.VehicleVar(routing.NodeToIndex(pair_revisit[0])) != routing.VehicleVar(routing.NodeToIndex(pair_revisit[1])))

In [37]:
###########
# Printer #
###########
def print_solution(data, routing, assignment):
  """Prints assignment on console"""
  # Inspect solution.
  time_dimension = routing.GetDimensionOrDie('Time')
  total_dist = 0
  time_matrix = 0
  visited_locations = []
  dropped_locations = list(range(data["num_locations"]))

  for vehicle_id in range(data["num_vehicles"]):
    index = routing.Start(vehicle_id)
    plan_output = 'Route for {0}, day {1}:\n'.format(
        data["dict_promotors_names"][int(vehicle_id/data["routed_days"])],
        vehicle_id-(int(vehicle_id/data["routed_days"])*data["routed_days"]))
    route_dist = 0
    while not routing.IsEnd(index):
      node_index = routing.IndexToNode(index)
      next_node_index = routing.IndexToNode(
        assignment.Value(routing.NextVar(index)))
      visited_locations.append(next_node_index)
      if next_node_index in dropped_locations:
        dropped_locations.remove(next_node_index)
      route_dist += routing.GetArcCostForVehicle(node_index, next_node_index, vehicle_id)
      time_var = time_dimension.CumulVar(index)
      time_min = assignment.Min(time_var)
      time_max = assignment.Max(time_var)
      plan_output += ' {0} Time({1},{2}) ->'.format(data["dict_place_names"][node_index], time_min, time_max)
      index = assignment.Value(routing.NextVar(index))

    node_index = routing.IndexToNode(index)
    time_var = time_dimension.CumulVar(index)
    route_time = assignment.Value(time_var)
    time_min = assignment.Min(time_var)
    time_max = assignment.Max(time_var)
    total_dist += route_dist
    time_matrix += route_time
    plan_output += ' {0} Time({1},{2})\n'.format(data["dict_place_names"][node_index], time_min, time_max)
    plan_output += 'Distance of the route: {0} m\n'.format(route_dist)
    plan_output += 'Time of the route: {0} min\n'.format(route_time)
    print(plan_output)
  print('Total Distance of all routes: {0} m'.format(total_dist))
  print('Total Time of all routes: {0} min'.format(time_matrix))
  dropped_visits_list = list(set(range(data["num_locations"])) - set(visited_locations))
  print('Dropped visits: ', list(map(data["dict_place_names"].get, dropped_visits_list)))

In [38]:
########
# Main #
########
def main():
  """Entry point of the program"""
  config = build_config()
  pdv_df = import_pdv_file(config["PDV_file"])
  distance_matrix = create_distance_matrix(pdv_df, config["API_key"])
  data = create_data_model(distance_matrix, config)

  # Create Routing Model
  routing = pywrapcp.RoutingModel(data["num_locations"], data["num_vehicles"], 
                                  data["depot"], data["arrive_location"])
  # Define weight of each edge
  distance_callback = create_distance_callback(data)
  routing.SetArcCostEvaluatorOfAllVehicles(distance_callback)
  # Add Time constraint
  time_callback = create_time_callback(data)
  add_time_constraints(routing, data, time_callback)
  # Add Revisit constraint
  add_revisit_constraints(routing, data)
  # Setting first solution heuristic (cheapest addition).
  search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
  search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
  # Adding penalty costs to allow dropping visits.
  for i in range(len(data["dict_promotors_names"])-1, data["num_locations"]):
    routing.AddDisjunction([routing.NodeToIndex(i)], data["penalty"])
    
  # Solve the problem.
  assignment = routing.SolveWithParameters(search_parameters)
  if assignment:
    printer = print_solution(data, routing, assignment)

In [39]:
%%time
main()

Route for Promotor_1, day 0:
 Home P1 Time(0,2) -> PDV_4 Time(44,46) -> PDV_2 Time(93,95) -> Home P1 Time(98,100)
Distance of the route: 4418677 m
Time of the route: 98 min

Route for Promotor_1, day 1:
 Home P1 Time(0,2) -> PDV_5 Time(44,46) -> Home P1 Time(98,100)
Distance of the route: 4416809 m
Time of the route: 98 min

Route for Promotor_1, day 2:
 Home P1 Time(0,100) -> Home P1 Time(0,100)
Distance of the route: 0 m
Time of the route: 0 min

Route for Promotor_1, day 3:
 Home P1 Time(0,100) -> Home P1 Time(0,100)
Distance of the route: 0 m
Time of the route: 0 min

Route for Promotor_1, day 4:
 Home P1 Time(0,100) -> Home P1 Time(0,100)
Distance of the route: 0 m
Time of the route: 0 min

Route for Promotor_2, day 0:
 Home P2 Time(0,100) -> Home P2 Time(0,100)
Distance of the route: 0 m
Time of the route: 0 min

Route for Promotor_2, day 1:
 Home P2 Time(0,100) -> Home P2 Time(0,100)
Distance of the route: 0 m
Time of the route: 0 min

Route for Promotor_2, day 2:
 Home P2 Time(